In [ ]:
!pip install --upgrade -q accelerate bitsandbytes

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
import torch
from PIL import Image
from transformers import pipeline

In [ ]:
class ImageDescriptionGenerator:
    """
    Class to generate textual descriptions from images using a pre-trained model.
    """

    def __init__(self, model_id):
        """
        Initialize the ImageDescriptionGenerator with the specified model ID.

        Args:
            model_id (str): The identifier of the model to use.
        """
        self.model_id = model_id
        self.quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )
        self.processor = None
        self.model = None
        self.pipe = None

    def load_model(self):
        """
        Load the model and processor from the specified model ID.

        Returns:
            AutoProcessor: The processor for the model.
            LlavaForConditionalGeneration: The loaded model.
        """
        self.pipe = pipeline("image-to-text", model=self.model_id, model_kwargs={"quantization_config": self.quantization_config})

    def generate_description(self, image_path):
        """
        Generate a textual description of the image using the pre-trained model.

        Args:
            image_path (str): The path to the image file.

        Returns:
            str: The textual description of the image.
        """
        image = Image.open(image_path)
        text = 'What are the objects present in the image please ignore the text? At the end give list of objects which are most highlighted in the image.'
        # text = 'Please provide a detailed description of the objects present in the image while disregarding any textual elements. Afterward, list the objects that are most prominent or highlighted within the image. Ensure that the response follows this structure: first, describe the image, highlighting key features; then, provide a list of highlighted objects based on their significance or visibility in the visual context. USER:  <input> ASSISTANT: <description>List of objects:<list>'
        prompt = f"USER: <image>\n{text}\nASSISTANT:"
        outputs = self.pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
        image_des = outputs[0]["generated_text"]
        return image_des

In [ ]:
def parse_image_description(image_description):
    # Find the index of "ASSISTANT" in the string
    assistant_index = image_description.find("ASSISTANT:")
    if assistant_index == -1:
        return None, None

    # Extract the text after "ASSISTANT"
    assistant_text = image_description[assistant_index + len("ASSISTANT:"):].strip()

    # Split the text into lines
    lines = assistant_text.split("\n")

    # Find the index where the list of objects starts
    objects_index = None
    for i, line in enumerate(lines):
        if "List of objects:" in line:
            objects_index = i
            break

    if objects_index is None:
        return assistant_text, None

    # Extract the list of objects
    objects_text = "\n".join(lines[objects_index + 1:])

    # Remove leading and trailing whitespaces
    objects_text = objects_text.strip()

    # Remove duplicates from the assistant_text
    assistant_text_lines = assistant_text.split('\n')
    objects_lines = objects_text.split('\n')

    # Remove duplicates from the assistant_text
    unique_lines = [line for line in assistant_text_lines if line not in objects_lines]

    # Join the unique lines back into a string
    unique_string = '\n'.join(unique_lines)

    return unique_string.strip().splitlines()[0], objects_text.strip()

In [ ]:
model_id = "llava-hf/llava-1.5-7b-hf"
generator = ImageDescriptionGenerator(model_id)
generator.load_model()

In [ ]:
image_path = '2.png'
description = generator.generate_description(image_path)

In [ ]:
description, objects = parse_image_description(description)
print(description)
print()
print(objects)